In [1]:
import os
import warnings
import docx

import numpy as np
import pandas as pd

from utils_prediction.model_evaluation import StandardEvaluator

pd.set_option('display.max_rows', 500)
warnings.filterwarnings('ignore')

In [2]:
evaluator = StandardEvaluator()

In [3]:
## Paths
base_artifacts_fpath = "/hpf/projects/lsung/projects/mimic4ds/Experiments/baseline/artifacts"
gen_artifacts_fpath = "/hpf/projects/lsung/projects/mimic4ds/Experiments/domain_gen/artifacts"
adapt_artifacts_fpath = "/hpf/projects/lsung/projects/mimic4ds/Experiments/domain_adapt/artifacts"

In [4]:
def get_metrics(task, thresh, df, n, prevalence):
    
    out = pd.DataFrame(columns = ['Task','Threshold','Sensitivity','Specificity','PPV','NPV','Total Positive Outcome','Total Negative Outcome', 'TP', 'FP','TN','FN'])
    
    pos = np.round(n*prevalence,0)
    neg = n - pos
    
    sensitivity = df.reset_index().loc[
        df.reset_index()['metric']=='sensitivity',
        'mean'
    ].values[0]
    
    specificity = df.reset_index().loc[
        df.reset_index()['metric']=='specificity',
        'mean'
    ].values[0]
    
    tp = np.round(pos * sensitivity,0)
    fn = pos - tp
    
    tn = np.round(neg * specificity,0)
    fp = neg - tn
    
    ppv = tp/(tp+fp)
    npv = tn/(tn+fn)
    
    out.loc[1,:] = [
        task, 
        thresh,
        np.round(sensitivity,2), 
        np.round(specificity,2),
        np.round(ppv,2),
        np.round(npv,2),
        pos,
        neg,
        tp,
        fp,
        tn,
        fn
    ]
    
    return out

def get_erm_predictive_values(artifacts_fpath, task, thresh, prevalence,metrics = ['sensitivity','specificity'], n=100, phase='test'):
    """
    Get average PPV & NPV of baseline model (2008 - 2010) in 2017 - 2019 data
    """
    fpath = os.path.join(
        artifacts_fpath,
        f"analysis_id={task}",
        "results/pred_probs",
        f"nn_erm.csv"
    )

    df = pd.read_csv(fpath)
    df['threshold'] = thresh


    # get ppv & npv
    df_results = evaluator.evaluate(df, strata_vars=['phase','iter'])
    
    df_results = df_results.query(
        "metric==@metrics and phase==@phase"
    ).groupby('metric')['performance'].agg(['mean','std'])
    
    out = get_metrics(task, thresh, df_results, n, prevalence)
    
    return out 

def get_adapt_predictive_values(artifacts_fpath, task, train_method, n_ood, thresh, prevalence,metrics = ['sensitivity','specificity'], n=100,phase='test'):
    fpath = os.path.join(
        artifacts_fpath,
        f"analysis_id={task}",
        "results/pred_probs",
        f"nn_{train_method}_{n_ood}.csv"
    )

    df = pd.read_csv(fpath)
    df['threshold'] = thresh


    # get ppv & npv
    df_results = evaluator.evaluate(df, strata_vars=['phase','iter'])
    
    df_results = df_results.query(
        "metric==@metrics and phase==@phase"
    ).groupby('metric')['performance'].agg(['mean','std'])
    
    out = get_metrics(task, thresh, df_results, n, prevalence)
    
    return out 


def get_base_predictive_values(artifacts_fpath, task, train_group, eval_group, thresh, prevalence, metrics = ['sensitivity','specificity'], n=100, phase='test'):
    """
    Get average PPV & NPV of baseline model (2008 - 2010) in 2017 - 2019 data
    """
    if phase == 'val':
        fpath = os.path.join(
            artifacts_fpath,
            f"analysis_id={task}",
            "results/pred_probs",
            f"nn_{train_group}_{train_group}.csv"
        )

        df = pd.read_csv(fpath)
        df['threshold'] = thresh

        # get ppv & npv
        df_results = evaluator.evaluate(df, strata_vars=['phase','iter'])

        df_results = df_results.query(
            "metric==@metrics and phase==@phase"
        ).groupby('metric')['performance'].agg(['mean','std'])
        
    elif phase == 'test':
        fpath = os.path.join(
            artifacts_fpath,
            f"analysis_id={task}",
            "results/pred_probs",
            f"nn_{train_group}_{eval_group}.csv"
        )

        df = pd.read_csv(fpath)
        df['threshold'] = thresh

        # get ppv & npv
        df_results = evaluator.evaluate(df, strata_vars=['phase','iter'])

        df_results = df_results.query(
            "metric==@metrics and phase==@phase"
        ).groupby('metric')['performance'].agg(['mean','std'])

    out = get_metrics(task, thresh, df_results, n, prevalence)
    
    return out 

def get_oracle_predictive_values(artifacts_fpath, task, eval_group, thresh, prevalence, metrics = ['sensitivity','specificity'], n=100, phase='test'):
    fpath = os.path.join(
        artifacts_fpath,
        f"analysis_id={task}",
        "results/pred_probs",
        f"nn_{eval_group}_{eval_group}.csv"
    )

    df = pd.read_csv(fpath)
    df['threshold'] = thresh

    # get ppv & npv
    df_results = evaluator.evaluate(df, strata_vars=['phase','iter'])
    
    df_results = df_results.query(
        "metric==@metrics and phase==@phase"
    ).groupby('metric')['performance'].agg(['mean','std'])
    
    
    out = get_metrics(task, thresh, df_results, n, prevalence)
    
    return out

def write_to_docx(df, fname='blank'):
    fpath = os.path.join(
        "/hpf/projects/lsung/projects",
        "mimic4ds/Experiments/domain_adapt/tables",
        f"{fname}.docx"
    )

    doc = docx.Document(fpath)

    t = doc.add_table(df.shape[0], df.shape[1])

    for i in range(df.shape[0]):
        for j in range(df.shape[-1]):
            t.cell(i,j).text = str(df.values[i,j])

    doc.save(fpath)

In [5]:
metrics = ['sensitivity','specificity']
task = 'sepsis'
train_group = '2008 - 2010'
eval_group = '2017 - 2019'
prevalence = 0.11
phase = 'test'

df_results = {}
for phase in ['test']:
    base_expected_metrics = pd.DataFrame()
    base_metrics = pd.DataFrame()
    oracle_metrics = pd.DataFrame()
    erm_metrics = pd.DataFrame()
    adapt_metrics = pd.DataFrame()
    for thresh in [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45]:
        base_expected_metrics = pd.concat((
            base_expected_metrics,
            get_base_predictive_values(base_artifacts_fpath, task, train_group, train_group, thresh, prevalence, phase=phase)
        ), axis = 0)
        
        base_metrics = pd.concat((
            base_metrics,
            get_base_predictive_values(base_artifacts_fpath, task, train_group, eval_group, thresh, prevalence, phase=phase)
        ), axis = 0)

        oracle_metrics = pd.concat((
            oracle_metrics,
            get_oracle_predictive_values(base_artifacts_fpath, task, eval_group, thresh, prevalence, phase=phase)
        ), axis = 0)

        erm_metrics = pd.concat((
            erm_metrics,
            get_erm_predictive_values(gen_artifacts_fpath, task, thresh, prevalence, phase=phase)
        ), axis = 0)

        adapt_metrics = pd.concat((
            adapt_metrics,
            get_adapt_predictive_values(adapt_artifacts_fpath, task, 'al_layer', 1500, thresh, prevalence, phase=phase)
        ), axis = 0)

        base_expected_metrics['model'] = 'Baseline [08-10](08-10)'
        base_metrics['model'] = 'Baseline [08-10]'
        oracle_metrics['model'] = 'Oracle [17-19]'
        erm_metrics['model'] = 'ERM [08-16]'
        adapt_metrics['model'] = 'DG/UDA [08-16]'

    df = pd.concat((
        base_expected_metrics,
        base_metrics,
        oracle_metrics,
        erm_metrics,
        adapt_metrics
    ),axis=0,ignore_index=True)

    df_out = pd.DataFrame()

    for x in ['Sensitivity','Specificity','PPV','NPV','TP','TN','FP','FN']:

        idf = df.pivot(
            columns='Threshold',
            index='model',
            values=x
        )

        idf['Metric'] = x
        df_out = pd.concat((df_out,idf))

    df_results[phase] = df_out.reset_index().set_index(['Metric','model'])

df_results['test']=df_results['test'].reindex(
    labels = [
        'Baseline [08-10](08-10)',
        'Baseline [08-10]',
        'ERM [08-16]',
        'DG/UDA [08-16]',
        'Oracle [17-19]',
    ], 
    level=1
)

df = df_results['test'].query("Metric==['Sensitivity','Specificity','PPV','NPV']")
df_c = df_results['test'].query("Metric==['TP','TN','FP','FN']")
df = pd.concat((df,df_c))
write_to_docx(df,f'clin_imp_{task}')

In [6]:
df

Threshold                             0.05   0.10   0.15   0.20   0.25   0.30  \
Metric      model                                                               
Sensitivity Baseline [08-10](08-10)   0.87   0.65   0.56   0.48   0.41   0.37   
            Baseline [08-10]          0.85   0.57   0.41   0.30   0.23   0.18   
            ERM [08-16]               0.90   0.76   0.62   0.53   0.45   0.37   
            DG/UDA [08-16]            0.90   0.75   0.62   0.53   0.44   0.36   
            Oracle [17-19]            0.91   0.61   0.43   0.33   0.25   0.20   
Specificity Baseline [08-10](08-10)   0.40   0.72   0.84   0.90   0.93   0.95   
            Baseline [08-10]          0.35   0.64   0.76   0.83   0.86   0.89   
            ERM [08-16]               0.42   0.68   0.79   0.85   0.89   0.92   
            DG/UDA [08-16]            0.42   0.68   0.79   0.85   0.89   0.92   
            Oracle [17-19]            0.30   0.74   0.86   0.91   0.94   0.96   
PPV         Baseline [08-10](08-10)   0.16   0.22   0.30   0.36   0.45   0.50   
            Baseline [08-10]          0.13   0.16   0.19   0.17   0.20   0.17   
            ERM [08-16]               0.16   0.22   0.27   0.32   0.33   0.36   
            DG/UDA [08-16]            0.16   0.22   0.27   0.32   0.36   0.36   
            Oracle [17-19]            0.14   0.23   0.28   0.33   0.38   0.40   
NPV         Baseline [08-10](08-10)   0.97   0.94   0.94   0.93   0.93   0.92   
            Baseline [08-10]          0.94   0.92   0.92   0.90   0.91   0.90   
            ERM [08-16]               0.97   0.95   0.95   0.94   0.93   0.92   
            DG/UDA [08-16]            0.97   0.95   0.95   0.94   0.93   0.92   
            Oracle [17-19]            0.96   0.94   0.93   0.92   0.91   0.91   
TP          Baseline [08-10](08-10)  10.00   7.00   6.00   5.00   5.00   4.00   
            Baseline [08-10]          9.00   6.00   5.00   3.00   3.00   2.00   
            ERM [08-16]              10.00   8.00   7.00   6.00   5.00   4.00   
            DG/UDA [08-16]           10.00   8.00   7.00   6.00   5.00   4.00   
            Oracle [17-19]           10.00   7.00   5.00   4.00   3.00   2.00   
TN          Baseline [08-10](08-10)  36.00  64.00  75.00  80.00  83.00  85.00   
            Baseline [08-10]         31.00  57.00  68.00  74.00  77.00  79.00   
            ERM [08-16]              37.00  60.00  70.00  76.00  79.00  82.00   
            DG/UDA [08-16]           37.00  61.00  70.00  76.00  80.00  82.00   
            Oracle [17-19]           27.00  66.00  76.00  81.00  84.00  86.00   
FP          Baseline [08-10](08-10)  53.00  25.00  14.00   9.00   6.00   4.00   
            Baseline [08-10]         58.00  32.00  21.00  15.00  12.00  10.00   
            ERM [08-16]              52.00  29.00  19.00  13.00  10.00   7.00   
            DG/UDA [08-16]           52.00  28.00  19.00  13.00   9.00   7.00   
            Oracle [17-19]           62.00  23.00  13.00   8.00   5.00   3.00   
FN          Baseline [08-10](08-10)   1.00   4.00   5.00   6.00   6.00   7.00   
            Baseline [08-10]          2.00   5.00   6.00   8.00   8.00   9.00   
            ERM [08-16]               1.00   3.00   4.00   5.00   6.00   7.00   
            DG/UDA [08-16]            1.00   3.00   4.00   5.00   6.00   7.00   
            Oracle [17-19]            1.00   4.00   6.00   7.00   8.00   9.00   

Threshold                             0.35   0.40   0.45  
Metric      model                                         
Sensitivity Baseline [08-10](08-10)   0.33   0.30   0.27  
            Baseline [08-10]          0.15   0.13   0.10  
            ERM [08-16]               0.29   0.21   0.07  
            DG/UDA [08-16]            0.29   0.20   0.07  
            Oracle [17-19]            0.15   0.10   0.06  
Specificity Baseline [08-10](08-10)   0.96   0.97   0.98  
            Baseline [08-10]          0.90   0.92   0.93  
            ERM [08-16]               0.95   0.97   0.99  
       

In [7]:
metrics = ['sensitivity','specificity']
task = 'longlos'
train_group = '2008 - 2010'
eval_group = '2017 - 2019'
prevalence = 0.31
phase = 'test'

df_results = {}
for phase in ['test']:
    base_expected_metrics = pd.DataFrame()
    base_metrics = pd.DataFrame()
    oracle_metrics = pd.DataFrame()
    erm_metrics = pd.DataFrame()
    adapt_metrics = pd.DataFrame()
    for thresh in [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7]:
        base_expected_metrics = pd.concat((
            base_expected_metrics,
            get_base_predictive_values(base_artifacts_fpath, task, train_group, train_group, thresh, prevalence, phase=phase)
        ), axis = 0)
        
        base_metrics = pd.concat((
            base_metrics,
            get_base_predictive_values(base_artifacts_fpath, task, train_group, eval_group, thresh, prevalence, phase=phase)
        ), axis = 0)

        oracle_metrics = pd.concat((
            oracle_metrics,
            get_oracle_predictive_values(base_artifacts_fpath, task, eval_group, thresh, prevalence, phase=phase)
        ), axis = 0)

        erm_metrics = pd.concat((
            erm_metrics,
            get_erm_predictive_values(gen_artifacts_fpath, task, thresh, prevalence, phase=phase)
        ), axis = 0)

        adapt_metrics = pd.concat((
            adapt_metrics,
            get_adapt_predictive_values(adapt_artifacts_fpath, task, 'coral', 500, thresh, prevalence, phase=phase)
        ), axis = 0)

        base_expected_metrics['model'] = 'Baseline [08-10](08-10)'
        base_metrics['model'] = 'Baseline [08-10]'
        oracle_metrics['model'] = 'Oracle [17-19]'
        erm_metrics['model'] = 'ERM [08-16]'
        adapt_metrics['model'] = 'DG/UDA [08-16]'

    df = pd.concat((
        base_expected_metrics,
        base_metrics,
        oracle_metrics,
        erm_metrics,
        adapt_metrics
    ),axis=0,ignore_index=True)

    df_out = pd.DataFrame()

    for x in ['Sensitivity','Specificity','PPV','NPV','TP','TN','FP','FN']:

        idf = df.pivot(
            columns='Threshold',
            index='model',
            values=x
        )

        idf['Metric'] = x
        df_out = pd.concat((df_out,idf))

    df_results[phase] = df_out.reset_index().set_index(['Metric','model'])

df_results['test']=df_results['test'].reindex(
    labels = [
        'Baseline [08-10](08-10)',
        'Baseline [08-10]',
        'ERM [08-16]',
        'DG/UDA [08-16]',
        'Oracle [17-19]',
    ], 
    level=1
)

df = df_results['test'].query("Metric==['Sensitivity','Specificity','PPV','NPV']")
df_c = df_results['test'].query("Metric==['TP','TN','FP','FN']")
df = pd.concat((df,df_c))
write_to_docx(df,f'clin_imp_{task}')

In [8]:
df

Threshold                             0.05   0.10   0.15   0.20   0.25   0.30  \
Metric      model                                                               
Sensitivity Baseline [08-10](08-10)   1.00   0.96   0.87   0.79   0.71   0.63   
            Baseline [08-10]          1.00   0.96   0.84   0.71   0.61   0.54   
            ERM [08-16]               1.00   0.97   0.91   0.83   0.75   0.67   
            DG/UDA [08-16]            1.00   0.97   0.92   0.84   0.76   0.68   
            Oracle [17-19]            1.00   0.99   0.97   0.93   0.84   0.73   
Specificity Baseline [08-10](08-10)   0.01   0.14   0.38   0.54   0.65   0.73   
            Baseline [08-10]          0.01   0.09   0.27   0.45   0.59   0.69   
            ERM [08-16]               0.01   0.12   0.30   0.46   0.58   0.67   
            DG/UDA [08-16]            0.01   0.11   0.28   0.45   0.57   0.66   
            Oracle [17-19]            0.00   0.05   0.15   0.28   0.42   0.55   
PPV         Baseline [08-10](08-10)   0.31   0.34   0.39   0.44   0.48   0.50   
            Baseline [08-10]          0.31   0.32   0.34   0.37   0.40   0.44   
            ERM [08-16]               0.31   0.33   0.36   0.41   0.44   0.48   
            DG/UDA [08-16]            0.31   0.33   0.36   0.41   0.43   0.48   
            Oracle [17-19]            0.31   0.32   0.34   0.37   0.39   0.43   
NPV         Baseline [08-10](08-10)   1.00   0.91   0.87   0.86   0.83   0.81   
            Baseline [08-10]          1.00   0.86   0.79   0.78   0.77   0.77   
            ERM [08-16]               1.00   0.89   0.87   0.86   0.83   0.82   
            DG/UDA [08-16]            1.00   0.88   0.86   0.86   0.83   0.82   
            Oracle [17-19]             NaN   1.00   0.91   0.91   0.85   0.83   
TP          Baseline [08-10](08-10)  31.00  30.00  27.00  25.00  22.00  19.00   
            Baseline [08-10]         31.00  30.00  26.00  22.00  19.00  17.00   
            ERM [08-16]              31.00  30.00  28.00  26.00  23.00  21.00   
            DG/UDA [08-16]           31.00  30.00  28.00  26.00  23.00  21.00   
            Oracle [17-19]           31.00  31.00  30.00  29.00  26.00  23.00   
TN          Baseline [08-10](08-10)   1.00  10.00  26.00  37.00  45.00  50.00   
            Baseline [08-10]          1.00   6.00  19.00  31.00  40.00  47.00   
            ERM [08-16]               1.00   8.00  20.00  32.00  40.00  46.00   
            DG/UDA [08-16]            1.00   7.00  19.00  31.00  39.00  46.00   
            Oracle [17-19]            0.00   3.00  10.00  20.00  29.00  38.00   
FP          Baseline [08-10](08-10)  68.00  59.00  43.00  32.00  24.00  19.00   
            Baseline [08-10]         68.00  63.00  50.00  38.00  29.00  22.00   
            ERM [08-16]              68.00  61.00  49.00  37.00  29.00  23.00   
            DG/UDA [08-16]           68.00  62.00  50.00  38.00  30.00  23.00   
            Oracle [17-19]           69.00  66.00  59.00  49.00  40.00  31.00   
FN          Baseline [08-10](08-10)   0.00   1.00   4.00   6.00   9.00  12.00   
            Baseline [08-10]          0.00   1.00   5.00   9.00  12.00  14.00   
            ERM [08-16]               0.00   1.00   3.00   5.00   8.00  10.00   
            DG/UDA [08-16]            0.00   1.00   3.00   5.00   8.00  10.00   
            Oracle [17-19]            0.00   0.00   1.00   2.00   5.00   8.00   

Threshold                             0.35   0.40   0.45   0.50   0.55   0.60  \
Metric      model                                                               
Sensitivity Baseline [08-10](08-10)   0.54   0.46   0.40   0.34   0.29   0.24   
            Baseline [08-10]          0.47   0.40   0.33   0.27   0.22   0.17   
            ERM [08-16]               0.58   0.50   0.43   0.35   0.28   0.21   
            DG/UDA [08-16]            0.59   0.51   0.43   0.36   0.29   0.22   
            Oracle [17-19]            0.63   0.53   0.45   0.37   0.30   0.19   
Specificity Baseline [08-10](0

In [9]:
metrics = ['sensitivity','specificity']
task = 'mortality'
train_group = '2008 - 2010'
eval_group = '2017 - 2019'
prevalence = 0.07
phase = 'test'

df_results = {}
for phase in ['test']:
    base_expected_metrics = pd.DataFrame()
    base_metrics = pd.DataFrame()
    oracle_metrics = pd.DataFrame()
    erm_metrics = pd.DataFrame()
    adapt_metrics = pd.DataFrame()
    for thresh in [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]:
        base_expected_metrics = pd.concat((
            base_expected_metrics,
            get_base_predictive_values(base_artifacts_fpath, task, train_group, train_group, thresh, prevalence, phase=phase)
        ), axis = 0)
        
        base_metrics = pd.concat((
            base_metrics,
            get_base_predictive_values(base_artifacts_fpath, task, train_group, eval_group, thresh, prevalence, phase=phase)
        ), axis = 0)

        oracle_metrics = pd.concat((
            oracle_metrics,
            get_oracle_predictive_values(base_artifacts_fpath, task, eval_group, thresh, prevalence, phase=phase)
        ), axis = 0)

        erm_metrics = pd.concat((
            erm_metrics,
            get_erm_predictive_values(gen_artifacts_fpath, task, thresh, prevalence, phase=phase)
        ), axis = 0)

        adapt_metrics = pd.concat((
            adapt_metrics,
            get_adapt_predictive_values(adapt_artifacts_fpath, task, 'coral', 500, thresh, prevalence, phase=phase)
        ), axis = 0)

        base_expected_metrics['model'] = 'Baseline [08-10](08-10)'
        base_metrics['model'] = 'Baseline [08-10]'
        oracle_metrics['model'] = 'Oracle [17-19]'
        erm_metrics['model'] = 'ERM [08-16]'
        adapt_metrics['model'] = 'DG/UDA [08-16]'

    df = pd.concat((
        base_expected_metrics,
        base_metrics,
        oracle_metrics,
        erm_metrics,
        adapt_metrics
    ),axis=0,ignore_index=True)

    df_out = pd.DataFrame()

    for x in ['Sensitivity','Specificity','PPV','NPV','TP','TN','FP','FN']:

        idf = df.pivot(
            columns='Threshold',
            index='model',
            values=x
        )

        idf['Metric'] = x
        df_out = pd.concat((df_out,idf))

    df_results[phase] = df_out.reset_index().set_index(['Metric','model'])

df_results['test']=df_results['test'].reindex(
    labels = [
        'Baseline [08-10](08-10)',
        'Baseline [08-10]',
        'ERM [08-16]',
        'DG/UDA [08-16]',
        'Oracle [17-19]',
    ], 
    level=1
)

df = df_results['test'].query("Metric==['Sensitivity','Specificity','PPV','NPV']")
df_c = df_results['test'].query("Metric==['TP','TN','FP','FN']")
df = pd.concat((df,df_c))
write_to_docx(df,f'clin_imp_{task}')

In [10]:
df

Threshold                             0.05   0.10   0.15   0.20   0.25   0.30  \
Metric      model                                                               
Sensitivity Baseline [08-10](08-10)   0.71   0.57   0.49   0.44   0.37   0.32   
            Baseline [08-10]          0.85   0.71   0.60   0.52   0.45   0.39   
            ERM [08-16]               0.84   0.71   0.62   0.55   0.49   0.44   
            DG/UDA [08-16]            0.84   0.71   0.62   0.55   0.49   0.44   
            Oracle [17-19]            0.88   0.72   0.61   0.54   0.48   0.42   
Specificity Baseline [08-10](08-10)   0.75   0.86   0.91   0.93   0.95   0.96   
            Baseline [08-10]          0.76   0.87   0.92   0.94   0.96   0.97   
            ERM [08-16]               0.78   0.88   0.92   0.94   0.95   0.96   
            DG/UDA [08-16]            0.78   0.88   0.92   0.94   0.95   0.96   
            Oracle [17-19]            0.78   0.88   0.93   0.95   0.96   0.97   
PPV         Baseline [08-10](08-10)   0.18   0.24   0.25   0.33   0.38   0.33   
            Baseline [08-10]          0.21   0.29   0.33   0.44   0.43   0.50   
            ERM [08-16]               0.23   0.29   0.33   0.40   0.43   0.50   
            DG/UDA [08-16]            0.23   0.29   0.33   0.40   0.43   0.50   
            Oracle [17-19]            0.23   0.31   0.36   0.44   0.50   0.50   
NPV         Baseline [08-10](08-10)   0.97   0.96   0.95   0.96   0.96   0.95   
            Baseline [08-10]          0.99   0.98   0.97   0.97   0.96   0.96   
            ERM [08-16]               0.99   0.98   0.97   0.97   0.96   0.96   
            DG/UDA [08-16]            0.99   0.98   0.97   0.97   0.96   0.96   
            Oracle [17-19]            0.99   0.98   0.97   0.97   0.96   0.96   
TP          Baseline [08-10](08-10)   5.00   4.00   3.00   3.00   3.00   2.00   
            Baseline [08-10]          6.00   5.00   4.00   4.00   3.00   3.00   
            ERM [08-16]               6.00   5.00   4.00   4.00   3.00   3.00   
            DG/UDA [08-16]            6.00   5.00   4.00   4.00   3.00   3.00   
            Oracle [17-19]            6.00   5.00   4.00   4.00   3.00   3.00   
TN          Baseline [08-10](08-10)  70.00  80.00  84.00  87.00  88.00  89.00   
            Baseline [08-10]         70.00  81.00  85.00  88.00  89.00  90.00   
            ERM [08-16]              73.00  81.00  85.00  87.00  89.00  90.00   
            DG/UDA [08-16]           73.00  81.00  85.00  87.00  89.00  90.00   
            Oracle [17-19]           73.00  82.00  86.00  88.00  90.00  90.00   
FP          Baseline [08-10](08-10)  23.00  13.00   9.00   6.00   5.00   4.00   
            Baseline [08-10]         23.00  12.00   8.00   5.00   4.00   3.00   
            ERM [08-16]              20.00  12.00   8.00   6.00   4.00   3.00   
            DG/UDA [08-16]           20.00  12.00   8.00   6.00   4.00   3.00   
            Oracle [17-19]           20.00  11.00   7.00   5.00   3.00   3.00   
FN          Baseline [08-10](08-10)   2.00   3.00   4.00   4.00   4.00   5.00   
            Baseline [08-10]          1.00   2.00   3.00   3.00   4.00   4.00   
            ERM [08-16]               1.00   2.00   3.00   3.00   4.00   4.00   
            DG/UDA [08-16]            1.00   2.00   3.00   3.00   4.00   4.00   
            Oracle [17-19]            1.00   2.00   3.00   3.00   4.00   4.00   

Threshold                             0.35   0.40   0.45   0.50  
Metric      model                                                
Sensitivity Baseline [08-10](08-10)   0.27   0.22   0.18   0.14  
            Baseline [08-10]          0.36   0.33   0.31   0.29  
            ERM [08-16]               0.39   0.34   0.30   0.25  
            DG/UDA [08-16]            0.39   0.35   0.31   0.26  
            Oracle [17-19]            0.38   0.35   0.31   0.29  
Specificity Baseline [08-10](08-10)   0.97   0.97   0.98   0.98  
            Baseline [08-10]          0.97   0.98   0.98   0.99  
   

In [11]:
metrics = ['sensitivity','specificity']
task = 'invasivevent'
train_group = '2008 - 2010'
eval_group = '2017 - 2019'
prevalence = 0.11
phase = 'test'

df_results = {}
for phase in ['test']:
    base_expected_metrics = pd.DataFrame()
    base_metrics = pd.DataFrame()
    oracle_metrics = pd.DataFrame()
    erm_metrics = pd.DataFrame()
    adapt_metrics = pd.DataFrame()
    for thresh in [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45]:
        base_expected_metrics = pd.concat((
            base_expected_metrics,
            get_base_predictive_values(base_artifacts_fpath, task, train_group, train_group, thresh, prevalence, phase=phase)
        ), axis = 0)
        
        base_metrics = pd.concat((
            base_metrics,
            get_base_predictive_values(base_artifacts_fpath, task, train_group, eval_group, thresh, prevalence, phase=phase)
        ), axis = 0)

        oracle_metrics = pd.concat((
            oracle_metrics,
            get_oracle_predictive_values(base_artifacts_fpath, task, eval_group, thresh, prevalence, phase=phase)
        ), axis = 0)

        erm_metrics = pd.concat((
            erm_metrics,
            get_erm_predictive_values(gen_artifacts_fpath, task, thresh, prevalence, phase=phase)
        ), axis = 0)

        adapt_metrics = pd.concat((
            adapt_metrics,
            get_adapt_predictive_values(adapt_artifacts_fpath, task, 'coral', 500, thresh, prevalence, phase=phase)
        ), axis = 0)

        base_expected_metrics['model'] = 'Baseline [08-10](08-10)'
        base_metrics['model'] = 'Baseline [08-10]'
        oracle_metrics['model'] = 'Oracle [17-19]'
        erm_metrics['model'] = 'ERM [08-16]'
        adapt_metrics['model'] = 'DG/UDA [08-16]'

    df = pd.concat((
        base_expected_metrics,
        base_metrics,
        oracle_metrics,
        erm_metrics,
        adapt_metrics
    ),axis=0,ignore_index=True)

    df_out = pd.DataFrame()

    for x in ['Sensitivity','Specificity','PPV','NPV','TP','TN','FP','FN']:

        idf = df.pivot(
            columns='Threshold',
            index='model',
            values=x
        )

        idf['Metric'] = x
        df_out = pd.concat((df_out,idf))

    df_results[phase] = df_out.reset_index().set_index(['Metric','model'])

df_results['test']=df_results['test'].reindex(
    labels = [
        'Baseline [08-10](08-10)',
        'Baseline [08-10]',
        'ERM [08-16]',
        'DG/UDA [08-16]',
        'Oracle [17-19]',
    ], 
    level=1
)

df = df_results['test'].query("Metric==['Sensitivity','Specificity','PPV','NPV']")
df_c = df_results['test'].query("Metric==['TP','TN','FP','FN']")
df = pd.concat((df,df_c))
write_to_docx(df,f'clin_imp_{task}')

In [12]:
df

Threshold                             0.05   0.10   0.15   0.20   0.25   0.30  \
Metric      model                                                               
Sensitivity Baseline [08-10](08-10)   0.82   0.66   0.53   0.45   0.42   0.40   
            Baseline [08-10]          0.92   0.81   0.75   0.68   0.63   0.59   
            ERM [08-16]               0.84   0.73   0.65   0.60   0.57   0.54   
            DG/UDA [08-16]            0.83   0.72   0.65   0.60   0.56   0.53   
            Oracle [17-19]            0.84   0.73   0.67   0.63   0.59   0.57   
Specificity Baseline [08-10](08-10)   0.62   0.84   0.91   0.94   0.96   0.97   
            Baseline [08-10]          0.47   0.77   0.88   0.92   0.94   0.96   
            ERM [08-16]               0.66   0.84   0.90   0.93   0.95   0.96   
            DG/UDA [08-16]            0.67   0.85   0.91   0.94   0.95   0.96   
            Oracle [17-19]            0.74   0.88   0.93   0.95   0.96   0.97   
PPV         Baseline [08-10](08-10)   0.21   0.33   0.43   0.50   0.56   0.57   
            Baseline [08-10]          0.18   0.31   0.42   0.53   0.58   0.60   
            ERM [08-16]               0.23   0.36   0.44   0.54   0.60   0.67   
            DG/UDA [08-16]            0.24   0.36   0.47   0.54   0.60   0.67   
            Oracle [17-19]            0.28   0.42   0.50   0.58   0.64   0.67   
NPV         Baseline [08-10](08-10)   0.96   0.95   0.94   0.93   0.93   0.92   
            Baseline [08-10]          0.98   0.97   0.96   0.96   0.95   0.94   
            ERM [08-16]               0.97   0.96   0.95   0.95   0.94   0.95   
            DG/UDA [08-16]            0.97   0.96   0.95   0.95   0.94   0.95   
            Oracle [17-19]            0.97   0.96   0.95   0.95   0.96   0.95   
TP          Baseline [08-10](08-10)   9.00   7.00   6.00   5.00   5.00   4.00   
            Baseline [08-10]         10.00   9.00   8.00   8.00   7.00   6.00   
            ERM [08-16]               9.00   8.00   7.00   7.00   6.00   6.00   
            DG/UDA [08-16]            9.00   8.00   7.00   7.00   6.00   6.00   
            Oracle [17-19]            9.00   8.00   7.00   7.00   7.00   6.00   
TN          Baseline [08-10](08-10)  55.00  75.00  81.00  84.00  85.00  86.00   
            Baseline [08-10]         42.00  69.00  78.00  82.00  84.00  85.00   
            ERM [08-16]              59.00  75.00  80.00  83.00  85.00  86.00   
            DG/UDA [08-16]           60.00  75.00  81.00  83.00  85.00  86.00   
            Oracle [17-19]           66.00  78.00  82.00  84.00  85.00  86.00   
FP          Baseline [08-10](08-10)  34.00  14.00   8.00   5.00   4.00   3.00   
            Baseline [08-10]         47.00  20.00  11.00   7.00   5.00   4.00   
            ERM [08-16]              30.00  14.00   9.00   6.00   4.00   3.00   
            DG/UDA [08-16]           29.00  14.00   8.00   6.00   4.00   3.00   
            Oracle [17-19]           23.00  11.00   7.00   5.00   4.00   3.00   
FN          Baseline [08-10](08-10)   2.00   4.00   5.00   6.00   6.00   7.00   
            Baseline [08-10]          1.00   2.00   3.00   3.00   4.00   5.00   
            ERM [08-16]               2.00   3.00   4.00   4.00   5.00   5.00   
            DG/UDA [08-16]            2.00   3.00   4.00   4.00   5.00   5.00   
            Oracle [17-19]            2.00   3.00   4.00   4.00   4.00   5.00   

Threshold                             0.35   0.40   0.45  
Metric      model                                         
Sensitivity Baseline [08-10](08-10)   0.39   0.37   0.35  
            Baseline [08-10]          0.56   0.55   0.52  
            ERM [08-16]               0.52   0.50   0.49  
            DG/UDA [08-16]            0.51   0.50   0.49  
            Oracle [17-19]            0.55   0.54   0.53  
Specificity Baseline [08-10](08-10)   0.98   0.98   0.98  
            Baseline [08-10]          0.96   0.97   0.97  
            ERM [08-16]               0.97   0.97   0.98  
       